In [2]:
# Importar las bibliotecas necesarias
import pandas as pd
from herramientas import obtener_resumen_valores_unicos
import numpy as np
from scipy.stats import pearsonr

In [3]:
# Leer los datos
df = pd.read_csv('data/train.csv', delimiter = '|')
df1 = pd.read_csv('data/RentabilidadProduct.csv', delimiter = '|')

# Descripción de los datos

## Data Entrenamiento [train.csv]
* **fecha_compra** Dia donde un cliente hizo una compra.
* **customer_id** Codigo de cliente
* **product_id** Codigo de producto
* **type_id** Tipo de negocio (GA: Gastronomia, PA: Panificacion, BO: Bodegas)
* **business_id** Codigo tipo de producto (a que negocio pertenece el producto)
* **channel_id** Canal de venta (A: Diadia, B: Insuma, V: Tradicional)
* **cantidad_venta** Cantidad de producto comprado
* **monto_venta_transf** Monto de compra
* **peso_venta_transf** Volumen de compra

## Data Producto [RentabilidadProduct.csv]
* **product_id** Codigo de producto
* **family_id** Codigo de familia (Marca a la que pertenece)
* **negocio_id** Codigo tipo de producto (a que negocio pertenece el producto)
* **category_id** Codigo de categoria de producto
* **tier_id** Codigo de tier, indica jerarquia de un producto ( 1 = premiun, 2 = mainstream, 3=economy, 4=super economy)
* **utilidad_bruta_transf** Utilidad bruta del producto

In [4]:
obtener_resumen_valores_unicos(df)

,Columna,Tipo de Dato,Valores Únicos,Total Valores,Valores Nulos
0,fecha_compra,object,327,2167848,0
1,customer_id,int64,15312,2167848,0
2,product_id,int64,1043,2167848,0
3,type_id,object,3,2167848,0
4,business_id,int64,10,2167848,0
5,channel_id,object,3,2167848,0
6,cantidad_venta,float64,147,2167848,0
7,monto_venta_transf,float64,36737,2167848,0
8,peso_venta_transf,float64,2382,2167848,0


In [5]:
obtener_resumen_valores_unicos(df1)

,Columna,Tipo de Dato,Valores Únicos,Total Valores,Valores Nulos
0,product_id,int64,937,937,0
1,family_id,int64,231,937,0
2,negocio_id,int64,3,937,0
3,category_id,int64,50,937,0
4,tier_id,int64,4,937,0
5,utilidad_bruta_transf,float64,925,937,0


# Filtrado colaborativo basado en usuarios (User-User)

# Matriz de calificaciones

In [6]:
# Agregamos un ranking
# Agrupar por cliente y producto, y contar la frecuencia de compra
frecuencia_compra = df.groupby(['customer_id', 'product_id']).size().reset_index(name='frecuencia')

# Normalizar el ranking entre 1 y 5
frecuencia_compra['ranking'] = frecuencia_compra['frecuencia'].apply(lambda x: 5 if x > 10 else (1 if x == 1 else (x-1)//2))

# Fusionar el ranking con el DataFrame original
df_ranking = pd.merge(df, frecuencia_compra[['customer_id', 'product_id', 'ranking']], on=['customer_id', 'product_id'], how='left')

In [7]:
df_ranking.head(3)

,fecha_compra,customer_id,product_id,type_id,business_id,channel_id,cantidad_venta,monto_venta_transf,peso_venta_transf,ranking
0,2022-01-31,7157,510,BO,21,V,1.0,0.153431,0.005211,2
1,2022-01-31,7157,512,BO,21,V,1.0,0.153431,0.005211,1
2,2022-01-31,9382,510,BO,21,V,1.0,0.153431,0.005211,0


In [8]:
# Crear la matriz de calificaciones usando pivot
df_matriz = df_ranking.drop_duplicates(subset=['customer_id', 'product_id'])
matriz_calificaciones = df_matriz.pivot(index='customer_id', columns='product_id', values='ranking')

# Rellenar los valores NaN con cero si es necesario
matriz_calificaciones = matriz_calificaciones.fillna(0)

# Ahora, matriz_calificaciones es un DataFrame que contiene la matriz de calificaciones que buscas


# Cálculo de Similitud entre Usuarios

## correlaciones de Pearson

In [14]:
# Asegúrate de que no haya valores faltantes
matriz_calificaciones = matriz_calificaciones.dropna()

# Convierte el DataFrame en una matriz NumPy
matriz_calificaciones_np = matriz_calificaciones.values

# Inicializa una matriz para almacenar los resultados de la correlación
pearson_similarity_matrix = np.zeros((matriz_calificaciones_np.shape[0], matriz_calificaciones_np.shape[0]))

# Bucle anidado para calcular la correlación entre las filas
for i in range(matriz_calificaciones_np.shape[0]):
    for j in range(matriz_calificaciones_np.shape[0]):
        if i != j:
            correlation, _ = pearsonr(matriz_calificaciones_np[i], matriz_calificaciones_np[j])
            pearson_similarity_matrix[i, j] = correlation

# Ahora 'pearson_similarity_matrix' contiene la correlación de Pearson entre las filas de tu DataFrame

/home/bbchanchis/recomienda/venv/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


## Similitud del Coseno

In [ ]:
# Calcular la similitud del coseno entre los usuarios
cosine_similarity_matrix = cosine_similarity(matriz_calificaciones)

print("Matriz de similitud del coseno:")
print(cosine_similarity_matrix)